In [11]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2023-09-27'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()


[*********************100%%**********************]  503 of 503 completed

3 Failed downloads:
['SOLV', 'GEV', 'VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1443499200, endDate = 1695787200")


In [43]:
# Calculate features and technical indicators per stock
# Garman-Klass Volatility
# RSI
# Bollinger Bands
# ATR
# MACD
# Dollar Volume

df['garman_klass_vol'] = (((np.log(df['high'])-np.log(df['low']))**2)/2)-(2*np.log(2)-1)*((np.log(df['adj close'])-(np.log(df['open']))**2))

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
                                                          
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.483551   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361618   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.651133   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.334900   52.790001   54.189999   51.880001   53.099998   
           ABT      33.478706   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.968475   89.519997   90.849998   89.500000   90.379997   
           YUM     122.811577  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.782722  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.547699  176.869995  178.449997  176.270004  176.580002   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0          3.419810        NaN       NaN   
           AAL       7478800.0          3.790025        NaN       NaN   
           AAPL    293461600.0          3.071580        NaN       NaN   
           ABBV     12842800.0          4.708104        NaN       NaN   
           ABT      12287500.0          3.847700        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0          6.102768  26.146750  4.488345   
           YUM       1500600.0          7.124523  36.057149  4.816569   
           ZBH       3610500.0          6.932117  31.893258  4.748085   
           ZBRA       355400.0          9.259367  29.494977  5.400991   
           ZTS       1463200.0          8.344044  42.623472  5.156108   

Price                bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2015-09-29 A            NaN       NaN       NaN       NaN      70.913550  
           AAL          NaN       NaN       NaN       NaN     279.420069  
           AAPL         NaN       NaN       NaN       NaN    7234.160810  
           ABBV         NaN       NaN       NaN       NaN     466.641852  
           ABT          NaN       NaN       NaN       NaN     411.369604  
...                     ...       ...       ...       ...            ...  
2023-09-26 XYL     4.570270  4.652196  0.033800 -2.159188     117.651912  
           YUM     4.846536  4.876502  0.142547 -1.363695     184.291052  
           ZBH     4.787752  4.827420 -0.381708 -0.881067     403.591519  
           ZBRA    5.539167  5.677342 -0.057389 -1.600791      79.595386  
           ZTS     5.214922  5.273735  0.651515 -1.188279     256.861393  

[988898 rows x 14 columns]

In [69]:
# Filter top 150 most liquid stocks for each month

last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()


In [70]:
# Calulate the 5 year rolling average dollar volume for each stock 

data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data[data['dollar_vol_rank'] < 150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

adj close  garman_klass_vol        rsi    bb_low  \
date       ticker                                                      
2016-10-31 AAL      39.134327          3.862275  62.203538  3.604673   
           AAPL     26.212471          3.065299  49.891040  3.294237   
           ABBV     40.245121          4.890794  27.477812  3.753446   
           ABT      34.293457          3.850304  38.008862  3.554632   
           ACN     103.569611          6.924762  53.823600  4.637342   
...                       ...               ...        ...       ...   
2023-09-30 WMT      53.783058          4.625217  54.722520  3.988367   
           XOM     114.292961          6.882600  59.440190  4.695106   
           MRNA     98.120003          6.369742  38.747314  4.582514   
           UBER     44.270000          4.101133  45.005268  3.806654   
           ABNB    132.279999          7.345376  44.494127  4.857047   

                     bb_mid   bb_high       atr      macd  
date       ticker                                          
2016-10-31 AAL     3.655493  3.706314  0.402199  1.131595  
           AAPL    3.323117  3.351997 -1.038688 -0.195978  
           ABBV    3.807610  3.861774 -0.893132 -0.760593  
           ABT     3.605106  3.655580 -1.035224 -0.650888  
           ACN     4.648980  4.660617 -0.996806 -0.135457  
...                     ...       ...       ...       ...  
2023-09-30 WMT     4.005838  4.023309 -0.196381  0.399458  
           XOM     4.735205  4.775304  0.601335  1.400623  
           MRNA    4.685332  4.788149 -0.529511 -0.376899  
           UBER    3.862227  3.917801 -0.746098 -0.133973  
           ABNB    4.940924  5.024801 -1.006939 -0.037854  

[12516 rows x 8 columns]

In [71]:
# Calculate monthly returns for different horizon as features

# Use lags for 1, 2, 3, 6, 9, 12 months gives 6 different lags

def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:
        
        df[f'return_{lag}m'] = (df['adj close']
                            .pct_change(lag)
                            .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                    upper=x.quantile(1-outlier_cutoff)))
                            .add(1)
                            .pow(1/lag)
                            .sub(1))
    return df
    
data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

dollar_volume   adj close  garman_klass_vol        rsi  \
date       ticker                                                           
2016-11-30 A           90.933919   41.481579          4.145110  44.135581   
           AAL        330.584399   44.876507          4.202543  71.397529   
           AAPL      3799.917813   25.646261          3.026905  48.015349   
           ABBV       342.438452   43.867039          5.100325  49.727654   
           ABT        302.588603   33.270939          3.812852  39.665419   
...                          ...         ...               ...        ...   
2023-09-30 CTVA       161.094953   50.462921          4.459005  44.013663   
           CARR       205.504044   52.174889          4.568475  42.987017   
           OTIS       171.241018   78.671936          5.751422  33.116217   
           ABNB       909.311388  132.279999          7.345376  44.494127   
           CEG        175.389873  108.026909          6.781284  55.245465   

                     bb_low    bb_mid   bb_high       atr      macd  \
date       ticker                                                     
2016-11-30 A       3.729552  3.772138  3.814724 -0.888157 -0.331613   
           AAL     3.656155  3.770906  3.885658  0.263773  2.344163   
           AAPL    3.249456  3.278521  3.307586 -1.011998 -0.314781   
           ABBV    3.722149  3.796301  3.870452 -0.617478 -0.257987   
           ABT     3.530306  3.566758  3.603210 -0.981659 -0.575622   
...                     ...       ...       ...       ...       ...   
2023-09-30 CTVA    3.914700  3.948840  3.982979 -0.803714 -0.385747   
           CARR    3.955645  4.025057  4.094468  1.818338 -1.120464   
           OTIS    4.374104  4.419394  4.464683 -1.028320 -1.534536   
           ABNB    4.857047  4.940924  5.024801 -1.006939 -0.037854   
           CEG     4.653660  4.693834  4.734007 -0.436215  0.366876   

                   dollar_vol_rank  return_1m  return_2m  return_3m  \
date       ticker                                                     
2016-11-30 A                 309.0   0.009410  -0.033586  -0.020946   
           AAL                73.0   0.146730   0.127700   0.086493   
           AAPL                1.0  -0.021601  -0.008718   0.015430   
           ABBV               67.0   0.089996  -0.013645  -0.014453   
           ABT                79.0  -0.029817  -0.048194  -0.030325   
...                            ...        ...        ...        ...   
2023-09-30 CTVA              290.0   0.005543  -0.049805  -0.038402   
           CARR              236.0  -0.082332  -0.059093   0.019790   
           OTIS              276.0  -0.073174  -0.064483  -0.036530   
           ABNB               37.0   0.005549  -0.067704   0.010603   
           CEG               265.0   0.041571   0.060899   0.059165   

                   return_6m  return_9m  return_12m  
date       ticker                                    
2016-11-30 A       -0.006214   0.019227    0.005104  
           AAL      0.065515   0.014886    0.010841  
           AAPL     0.018834   0.016838   -0.003828  
           ABBV    -0.002749   0.015097    0.006898  
           ABT     -0.004603   0.000125   -0.011648  
...                      ...        ...         ...  
2023-09-30 CTVA    -0.027278  -0.015197   -0.008915  
           CARR     0.025342   0.028590    0.034814  
           OTIS    -0.009024   0.002663    0.019569  
           ABNB     0.010289   0.048608    0.019401  
           CEG      0.056503   0.026987    0.023339  

[40703 rows x 16 columns]

In [111]:
# Download Fama-French factors and calculate rolling factor betas

factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                               'famafrench',
                               start='2010')[0].drop('RF', axis=1)

factor_data.index = factor_data.index.to_timestamp()

factor_data = factor_data.resample('M').last().div(100)

factor_data.index.name = 'date'

factor_data = factor_data.join(data['return_1m']).sort_index()

factor_data


Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2016-11-30 A       0.0486  0.0707  0.0821 -0.0018  0.0370   0.009410
           AAL     0.0486  0.0707  0.0821 -0.0018  0.0370   0.146730
           AAPL    0.0486  0.0707  0.0821 -0.0018  0.0370  -0.021601
           ABBV    0.0486  0.0707  0.0821 -0.0018  0.0370   0.089996
           ABT     0.0486  0.0707  0.0821 -0.0018  0.0370  -0.029817
...                   ...     ...     ...     ...     ...        ...
2023-09-30 XYL    -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.135407
           YUM    -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.041506
           ZBH    -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.055910
           ZBRA   -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.185630
           ZTS    -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.071597

[40703 rows x 6 columns]

In [117]:
# Filtering out stocks with less than 10 months of data, as will break our model

observations = factor_data.groupby(level=1).size()

valid_stocks = observations[observations >= 10]

factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]

factor_data



Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2016-11-30 A       0.0486  0.0707  0.0821 -0.0018  0.0370   0.009410
           AAL     0.0486  0.0707  0.0821 -0.0018  0.0370   0.146730
           AAPL    0.0486  0.0707  0.0821 -0.0018  0.0370  -0.021601
           ABBV    0.0486  0.0707  0.0821 -0.0018  0.0370   0.089996
           ABT     0.0486  0.0707  0.0821 -0.0018  0.0370  -0.029817
...                   ...     ...     ...     ...     ...        ...
2023-09-30 XYL    -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.135407
           YUM    -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.041506
           ZBH    -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.055910
           ZBRA   -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.185630
           ZTS    -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.071597

[40695 rows x 6 columns]

In [126]:
# Now calculate rolling factor betas

betas = (factor_data.groupby(level=1,
                            group_keys=False)
         .apply(lambda x: RollingOLS(endog=x['return_1m'], 
                                     exog=sm.add_constant(x.drop('return_1m', axis=1)),
                                     window=min(24, x.shape[0]),
                                     min_nobs=len(x.columns)+1)
         .fit(params_only=True)
         .params
         .drop('const', axis=1)))

betas

TypeError: window must be integer_like (int or np.integer, but not bool or timedelta64) or None